## Deliverable 2. Create a Customer Travel Destinations Map.

In [54]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import time
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.4560,-114.3525,40.41,97,20,6.91,few clouds
1,1,Urumqi,CN,43.8010,87.6005,40.15,56,0,6.71,clear sky
2,2,Hobart,AU,-42.8794,147.3294,60.12,97,75,1.01,broken clouds
3,3,Caravelas,BR,-17.7125,-39.2481,76.21,90,89,8.43,overcast clouds
4,4,Northam,GB,51.0333,-4.2167,59.40,89,100,8.93,overcast clouds


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp= float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
prefered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &  (city_data_df["Max Temp"] >= min_temp)]

In [16]:
# 4a. Determine if there are any empty rows.
prefered_cities_df.count()

Unnamed: 0             224
City                   224
Country                224
Lat                    224
Lng                    224
Max Temp               224
Humidity               224
Cloudiness             224
Wind Speed             224
Current Description    224
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
prefered_cities_df=prefered_cities_df.drop(prefered_cities_df.columns[0], axis=1) 

In [29]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = prefered_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = " "
hotel_df.head(100)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Caravelas,BR,76.21,overcast clouds,-17.7125,-39.2481,
5,Kapaa,US,87.78,light rain,22.0752,-159.3190,
10,Butaritari,KI,82.81,broken clouds,3.0707,172.7902,
12,Hilo,US,79.29,moderate rain,19.7297,-155.0900,
15,Georgetown,MY,87.73,broken clouds,5.4112,100.3354,
...,...,...,...,...,...,...,...
331,Sinnamary,GF,79.23,scattered clouds,5.3833,-52.9500,
332,Makakilo City,US,82.72,broken clouds,21.3469,-158.0858,
337,Merauke,ID,79.25,light rain,-8.4667,140.3333,
343,Vila Velha,BR,76.95,few clouds,-20.3297,-40.2925,


In [41]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {"radius": 5000, "type": "lodging", "key": g_key}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
 
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [68]:
hotel_df.loc[hotel_df["Hotel Name"]== " "]= np.nan 
hotel_df=hotel_df.dropna(axis=0)
len(hotel_df)

212

In [69]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [71]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd> {Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [72]:
# 11a. Add a marker layer for each city to the map. 
#Add a heatmap of temperature for the vacation spots and marker for each city
locations = hotel_df[["Lat","Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

#call the figure to plot the data
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))